In [3]:
import os
import json
import urllib.request
import h5py
import numpy as np
import json
import matplotlib.pyplot as plt
import pickle

from IPython.display import Image,display,clear_output

In [9]:
import keras 
from keras.models import Sequential,load_model
from keras.applications.vgg16 import VGG16
from keras.applications.imagenet_utils import preprocess_input,decode_predictions   #?
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img
from keras.utils.data_utils import get_file

## Input

In [7]:
first_gate = VGG16(weights='imagenet')

In [ ]:
second_gate = load_model('data1a/ft_model.h5')
location_model = load_model('data2a/ft_model.h5')
severity_model = load_model('data3a/ft_model.h5')

In [5]:
with open('vgg16_cat_list.pk','rb') as f:
    cat_list = pickle.load(f)

In [6]:
cat_list

[('n03770679', 'minivan'),
 ('n04285008', 'sports_car'),
 ('n03100240', 'convertible'),
 ('n02814533', 'beach_wagon'),
 ('n03930630', 'pickup'),
 ('n04461696', 'tow_truck'),
 ('n03670208', 'limousine'),
 ('n04037443', 'racer'),
 ('n02974003', 'car_wheel'),
 ('n02930766', 'cab'),
 ('n03769881', 'minibus'),
 ('n03796401', 'moving_van'),
 ('n03594945', 'jeep'),
 ('n03459775', 'grille'),
 ('n03445924', 'golfcart'),
 ('n03791053', 'motor_scooter'),
 ('n04252225', 'snowplow'),
 ('n03977966', 'police_van'),
 ('n02965783', 'car_mirror'),
 ('n03417042', 'garbage_truck'),
 ('n02701002', 'ambulance'),
 ('n04252077', 'snowmobile'),
 ('n04467665', 'trailer_truck'),
 ('n03478589', 'half_track'),
 ('n03891332', 'parking_meter'),
 ('n03777568', 'Model_T'),
 ('n04065272', 'recreational_vehicle'),
 ('n03127747', 'crash_helmet'),
 ('n03649909', 'lawn_mower'),
 ('n02704792', 'amphibian'),
 ('n04517823', 'vacuum'),
 ('n02917067', 'bullet_train'),
 ('n04465501', 'tractor'),
 ('n03785016', 'moped'),
 ('n0377

## First Gate Setup - Car or Not?

In [8]:
# from Keras GitHub  
CLASS_INDEX = None
CLASS_INDEX_PATH = 'https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json'

def get_predictions(preds, top=5):
    global CLASS_INDEX
    if len(preds.shape) != 2 or preds.shape[1] != 1000:
        raise ValueError('`decode_predictions` expects '
                         'a batch of predictions '
                         '(i.e. a 2D array of shape (samples, 1000)). '
                         'Found array with shape: ' + str(preds.shape))
    if CLASS_INDEX is None:
        fpath = get_file('imagenet_class_index.json',
                         CLASS_INDEX_PATH,
                         cache_subdir='models')
        CLASS_INDEX = json.load(open(fpath))
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [13]:
def prepare_img_244(img_path):
    #urllib.request.urlretrieve(img_url,'save.jpg')
    img = load_img(img_path,target_size=(224,224))
    x = img_to_array(img)
    x = np.expand_dims(x,axis=0)
    x = preprocess_input(x)
    return x

In [20]:
def car_categories_gate(img244,model):
    print("Validating that this is a picture of your car...")
    out = model.predict(img244)
    top = get_predictions(out,top=5)
    for j in top[0]:
        if j[0:2] in cat_list:
            # print j[0:2]
            return True # "Validation complete - proceed to damage evaluation"
    return False # "Are you sure this is a picture of your car? 
                 # Please take another picture (try a different angle or lighting) and try again."

## Second Gate Setup - Damaged or Whole?

In [33]:
def prepare_img_256(img_path):
    img = load_img(img_path,target_size=(256,256))
    x = img_to_array(img)
    x = x.reshape((1,)+x.shape)/255  
    return x

In [21]:
def car_damage_gate(img_256,model):
    print("Validating that damage exists...")
    pred = model.predict(img_256)
    if pred[0][0] <=.5:
        return True # print "Validation complete - proceed to location and severity determination"
    else:
        return False
        # print "Are you sure that your car is damaged? Please submit another picture of the damage."
        # print "Hint: Try zooming in/out, using a different angle or different lighting"

## Location and Severity Assessment

In [23]:
def location_assessment(img_256,model):
    print("Determining location of damage...")
    pred = model.predict(img_256)
    pred_label = np.argmax(pred,axis=1)
    d = {0: 'Front', 1: 'Rear', 2: 'Side'}
    for key in d.iterkeys():
        if pred_label[0] == key:
            print("Assessment: {} damage to vehicle".format(d[key]))
    print("Location assessment complete.")

In [27]:
def severity_assessment(img_256, model):
    print("Determining severity of damage...")
    pred = model.predict(img_256)
    pred_label = np.argmax(pred, axis=1)
    d = {0: 'Minor', 1: 'Moderate', 2: 'Severe'}
    for key in d.iterkeys():
        if pred_label[0] == key:
            print("Assessment: {} damage to vehicle".format(d[key]))
    print("Severity assessment complete.")

In [31]:
# load models
def engine():
    while True:
        try:
            print('Enter img url here(or type q or Q to quit)')
            img_url = input('Upload image file or enter image-url')
            if img_url in ['q','Q']:
                return None
            
            img_path,_ = urllib.request.urlretrieve(img_url,'save.jpg')
            img_244 = prepare_img_244(img_path)
            
            g1 = car_categories_gate(img_244,first_gate)
            display(Image('save.jpg', width=300))
            if g1 is False:
                print("Are you sure this is a picture of your car? Please submit another picture of the damage.")
                print("Hint: Try zooming in/out, using a different angle or different lighting.")
                break
            
            img_256 = prepare_img_256(img_path)
            g2 = car_damage_gate(img_256,second_gate)
            if g2 is False:
                print("Are you sure that your car is damaged? Please submit another picture of the damage.")
                print("Hint: Try zooming in/out, using a different angle or different lighting.")
                break
            
            x = location_assessment(img_256,location_model)
            y = severity_assessment(img_256,severity_model)
        except:
            print("Image not accessible. Please try again.")
            break